#Black Scholes Scenario Generator
Given an option with a strike price and days to expiry, this function generates the Black Scholes value of the option for a range of market prices on each day until expiry. 

Assumptions:

Constant volatility. Due to the volatility smile, the Black Scholes calculations become less accurate for the generated stock prices further from the current stock price. 

We will incorporate stochastic volatility in the future.




##Inputs:
Update then run all cells (CTRL+F9)


In [ ]:
## Range of market prices to generate
priceRangeMax = 3
step = 0.5       ## price increments

## "C" for Call ## "P" for Put
Type = "P"

## Current price
CurrentPrice = 339

## Strike price
Strike = 338

## Time to expiry (days) ## include current day and expiry day 
ttDays = 1

## Volatility (annual)   
ssigma = 0.35

## Risk free rate (annual continuous compounding)
rr = 0.04
## Dividend
DivBoolean = FALSE   ## true if there is dividend before expiry date. False otherwise
Div = NULL  ## vector of dividends
## Time to ex-dividend date (years)
ttDiv = NULL  ## vector of times 


In [ ]:
#@title
## No dividends
NoDiv = function(S, K, t, sigma, r, type="C"){
  d1 = (log(S/K) + (r + sigma**2/2)*t)/(sigma*sqrt(t)) 
  d2 = d1 - sigma*sqrt(t)
  price = ifelse(type == "P", 
                 pnorm(d2, lower.tail = F)*K*exp(-r*t) - pnorm(d1, lower.tail = F)*S,
                 pnorm(d1)*S - pnorm(d2)*K*exp(-r*t))
  delta = ifelse(type == "P", pnorm(d1)-1, pnorm(d1))
  output = c(price, delta)
  names(output) = c("Price", "Delta")
  return(round(output,4))
}

## Black's approximation for American options with discrete dividends
## Option price = max(method 1, method 2)

BlackApprox = function(S, K , t, sigma, r, div, tdiv, type="C"){
  
  ## 1: European option with stock price reduced by the present value of dividends
  price = c()
  delta = c()
  PVdiv = div %*% exp(-tdiv*r)
  Sdiv = S - PVdiv
  d1 = (log(Sdiv/K) + (r + sigma**2/2)*t)/(sigma*sqrt(t)) 
  d2 = d1 - sigma*sqrt(t)
  price[1] = ifelse(type == "P", 
                    pnorm(d2, lower.tail = F)*K*exp(-r*t) - pnorm(d1, lower.tail = F)*Sdiv,
                    pnorm(d1)*Sdiv - pnorm(d2)*K*exp(-r*t))
  delta[1] = ifelse(type == "P", pnorm(d1)-1, pnorm(d1))
  
  ## 2: European option that expires on the day before the last dividend
  ndiv = length(div)
  PVdiv = div[-ndiv] %*% exp(-tdiv[-ndiv]*r) 
  Sdiv = S - PVdiv
  t = tdiv[ndiv]
  d1 = (log(Sdiv/K) + (r + sigma**2/2)*t)/(sigma*sqrt(t))
  d2 = d1 - sigma*sqrt(t)
  price[2] = ifelse(type == "P", 
                    pnorm(d2, lower.tail = F)*K*exp(-r*t) - pnorm(d1, lower.tail = F)*Sdiv,
                    pnorm(d1)*Sdiv - pnorm(d2)*K*exp(-r*t))
  delta[2] = ifelse(type == "P", pnorm(d1)-1, pnorm(d1))
  
  ## Take max of both methods
  output = c(price[which.max(price)], delta[which.max(price)])
  
  names(output) = c("Price", "Delta")
  return(round(output,4))
}

BlackScholes = function(S, K, t, sigma, r, divBoolean=F, div=NULL, tdiv=NULL, type="C"){
  if(divBoolean){
    BlackApprox(S,K,t,sigma,r,div,tdiv,type)
  } else {
    NoDiv(S,K,t,sigma,r,type)
  }
}

BS_Scenario = function(){
  priceRange = round(seq(CurrentPrice-priceRangeMax,  
                         CurrentPrice+priceRangeMax, 
                         by = step),1)
  prices = matrix(nrow=ttDays, ncol = length(priceRange))
  ii = 1
  jj = 1
  for (day in ttDays:1){
    for (price in priceRange){
      prices[ii, jj] = BlackScholes(S = price,
                                    K = Strike,
                                    t = day/365,
                                    sigma = ssigma,
                                    r = rr,
                                    divBoolean = DivBoolean,
                                    div = Div,
                                    tdiv = ttDiv,
                                    type = Type)[1]
      jj = ifelse(jj == length(priceRange), 1, jj+1)
    }
    ii = ifelse(ii == ttDays, 1, ii+1)
  }
  colnames(prices) = priceRange
  rownames(prices) = ttDays:1
  if(Type=="P"){
    print("Put Option")
  } else {
    print("Call Option")
  }
  print(paste('Strike               :', Strike))
  print(paste('Days to Expiry       :' , ttDays))
  print(paste('Current Option Value :', (BlackScholes(S = CurrentPrice,
                                                     K = Strike,
                                                     t = ttDays/365,
                                                     sigma = ssigma,
                                                     r = rr,
                                                     divBoolean = DivBoolean,
                                                     div = Div,
                                                     tdiv = ttDiv,
                                                     type = Type)[1])))
  print(paste('Current Stock Price  :', CurrentPrice))
  print("--------------------------------------------")
  print("ROW: DAYS TO EXPIRY #### COLUMN: STOCK PRICE")
  print("1 is DAY OF EXPIRY")
  print("--------------------------------------------")

  return(round(prices,2))
}

In [ ]:
BS_Scenario()

[1] "Put Option"
[1] "Strike               : 338"
[1] "Days to Expiry       : 1"
[1] "Current Option Value : 1.9898"
[1] "Current Stock Price  : 339"
[1] "--------------------------------------------"
[1] "ROW: DAYS TO EXPIRY #### COLUMN: STOCK PRICE"
[1] "1 is DAY OF EXPIRY"
[1] "--------------------------------------------"


,336,336.5,337,337.5,338,338.5,339,339.5,340,340.5,341,341.5,342
1,3.57,3.26,2.98,2.71,2.45,2.21,1.99,1.78,1.59,1.41,1.25,1.1,0.97
